In [ ]:
# =====================================================
# FAKE NEWS DETECTION – MiniLM + LIAR
# =====================================================

# 1. CÀI ĐẶT & IMPORT
!pip install -q transformers datasets torch scikit-learn pandas numpy psutil accelerate

import os, re, time, shutil, psutil, warnings
import pandas as pd
import numpy as np
import torch
from datasets import load_dataset, Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight
from transformers import (
    BertTokenizerFast,
    BertForSequenceClassification,
    Trainer, TrainingArguments, EarlyStoppingCallback, set_seed,
    DataCollatorWithPadding
)
from google.colab import drive
from tqdm.auto import tqdm

warnings.filterwarnings("ignore")

# 2. KIỂM TRA GPU
if torch.cuda.is_available():
    device_name = torch.cuda.get_device_name(0)
    vram_gb = torch.cuda.get_device_properties(0).total_memory / 1e9
else:
    device_name = "CPU"
    vram_gb = 0.0
print(f"Device: {device_name} | CUDA: {torch.cuda.is_available()} | VRAM: {vram_gb:.1f} GB")

# 3. MOUNT DRIVE & CHECK SPACE
drive.mount('/content/drive', force_remount=False)

def check_drive_space(path="/content/drive/MyDrive"):
    try:
        usage = psutil.disk_usage(path)
        used_gb = usage.used / 1e9
        total_gb = usage.total / 1e9
        pct = used_gb/total_gb*100 if total_gb>0 else 0
        print(f"Drive: {used_gb:.1f}GB / {total_gb:.1f}GB ({pct:.1f}%)")
        if pct > 90:
            print("CẢNH BÁO: Dung lượng Drive gần đầy!")
    except Exception as e:
        print("Không thể kiểm tra dung lượng Drive:", e)

check_drive_space()

# 4. OUTPUT_DIR + CHECKPOINT MANAGEMENT
OUTPUT_DIR = "/content/drive/MyDrive/WELFake_MiniLM_Pro"
os.makedirs(OUTPUT_DIR, exist_ok=True)

def manage_checkpoints(output_dir, keep_latest=2):
    ckpts = [c for c in os.listdir(output_dir) if c.startswith("checkpoint-")]
    if len(ckpts) <= keep_latest:
        return

    def idx_from_name(n):
        parts = n.split("-")
        try:
            return int(parts[-1])
        except:
            return os.path.getmtime(os.path.join(output_dir, n))

    ckpts_sorted = sorted(ckpts, key=idx_from_name)
    for old_ckpt in ckpts_sorted[:-keep_latest]:
        shutil.rmtree(os.path.join(output_dir, old_ckpt), ignore_errors=True)
        print(f"Đã xóa checkpoint cũ: {old_ckpt}")

def get_last_checkpoint(output_dir):
    ckpts = [c for c in os.listdir(output_dir) if c.startswith("checkpoint-")]
    if not ckpts:
        return None

    def idx_from_name(n):
        parts = n.split("-")
        try:
            return int(parts[-1])
        except:
            return os.path.getmtime(os.path.join(output_dir, n))

    ckpts_sorted = sorted(ckpts, key=idx_from_name, reverse=True)
    manage_checkpoints(output_dir, keep_latest=2)
    return os.path.join(output_dir, ckpts_sorted[0])

last_checkpoint = get_last_checkpoint(OUTPUT_DIR)
print("Checkpoint gần nhất:", last_checkpoint or "Không có → Train từ đầu")

# =====================================================
# 5. TẢI DATASET LIAR & CHUYỂN ĐỔI (MiniLM Version)
# =====================================================
print("\nĐang tải dataset LIAR (chengxuphd/liar2)...")

try:
    dataset = load_dataset("chengxuphd/liar2")
except Exception as e:
    print(f"Lỗi tải 'chengxuphd/liar2': {e}")
    dataset = load_dataset("liar")

# Chuyển sang Pandas để xử lý linh hoạt
df_train = pd.DataFrame(dataset['train'])
df_val   = pd.DataFrame(dataset['validation'])
df_test  = pd.DataFrame(dataset['test'])

# Gộp lại để xử lý nhất quán
df = pd.concat([df_train, df_val, df_test], ignore_index=True)
print(f"Tổng số mẫu dữ liệu: {len(df)}")

# -----------------------------------------------------
# A. MAPPING NHÃN (6 class -> 2 class)
# -----------------------------------------------------
# 0: false, 1: half-true, 2: mostly-true, 3: true, 4: barely-true, 5: pants-fire
def map_liar_labels(label):
    # Fake group: false(0), barely-true(4), pants-fire(5)
    if label in [0, 4, 5]:
        return 0
    # Real group: half-true(1), mostly-true(2), true(3)
    elif label in [1, 2, 3]:
        return 1
    return 0

print("Đang chuyển đổi nhãn sang Binary (0: Fake, 1: Real)...")
df['label'] = df['label'].apply(map_liar_labels)

# -----------------------------------------------------
# B. TẠO INPUT CONTENT (Feature Engineering)
# -----------------------------------------------------
# MiniLM có giới hạn input ngắn (thường tốt nhất < 384 tokens).
# Chúng ta cần chọn lọc thông tin ngữ cảnh đắt giá nhất để ghép vào.
def create_liar_content(row):
    stmt = str(row.get('statement', '')).strip()

    # Lấy Metadata
    speaker = str(row.get('speaker', ''))
    party   = str(row.get('party_affiliation', ''))
    subject = str(row.get('subject', ''))
    context = str(row.get('context', ''))

    # Cấu trúc: [Statement] [SEP] Speaker (Party) | Subject | Context
    # Ví dụ: "Tax cuts are good [SEP] donald-trump (republican) | taxes | debate"
    meta_info = f"{speaker} ({party}) | {subject} | {context}"

    return stmt + " [SEP] " + meta_info

print("Đang tạo nội dung đầu vào (Statement + Context)...")
df['content'] = df.apply(create_liar_content, axis=1)

# -----------------------------------------------------
# C. LÀM SẠCH VĂN BẢN
# -----------------------------------------------------
def clean_text(text):
    if not isinstance(text, str): return ""
    t = text.lower()
    t = re.sub(r'https?://\S+|www\.\S+', ' ', t)
    t = re.sub(r'<.*?>', ' ', t)
    # Giữ lại các ký tự có ý nghĩa ngữ pháp cho mô hình BERT học
    t = re.sub(r'[^a-zA-Z0-9\s\.\,\-\(\)\|\!]', ' ', t)
    t = re.sub(r'\s+', ' ', t).strip()
    return t

df['content'] = df['content'].apply(clean_text)
df = df[df['content'].str.len() > 10] # Bỏ câu quá ngắn
print(f" → Sau xử lý: {len(df):,}")

# Kiểm tra phân bố nhãn
label_counts = df['label'].value_counts(normalize=True)
print(f"Phân bố: Fake(0)={label_counts.get(0,0):.1%}, Real(1)={label_counts.get(1,0):.1%}")

# Tính Class Weights (Quan trọng vì LIAR mất cân bằng hơn WELFake)
classes = np.array([0,1])
class_weights = compute_class_weight('balanced', classes=classes, y=df['label'])
class_weight_dict = {0: float(class_weights[0]), 1: float(class_weights[1])}
print("Class weights:", class_weight_dict)

# =====================================================
# 6. SPLIT DATA
# =====================================================
labels = df['label'].values
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42, stratify=labels)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['label'])

print(f"Train: {len(train_df):,} | Val: {len(val_df):,} | Test: {len(test_df):,}")

train_dataset = Dataset.from_pandas(train_df[['content','label']].reset_index(drop=True))
val_dataset   = Dataset.from_pandas(val_df[['content','label']].reset_index(drop=True))
test_dataset  = Dataset.from_pandas(test_df[['content','label']].reset_index(drop=True))

dataset_dict = DatasetDict({"train": train_dataset, "validation": val_dataset, "test": test_dataset})

# =====================================================
# 7. TOKENIZER (SỬ DỤNG TÊN MODEL CHUẨN)
# =====================================================
# Thay vì bản L6 (bị lỗi), ta dùng bản chuẩn L12 của Microsoft
# Tên đúng: "microsoft/MiniLM-L12-H384-uncased"
MODEL_NAME = "microsoft/MiniLM-L12-H384-uncased"

tokenizer = BertTokenizerFast.from_pretrained(MODEL_NAME)

def tokenize_fn(batch):
    return tokenizer(batch["content"], truncation=True, max_length=384, padding=False)

print(f"Đang tải tokenizer từ: {MODEL_NAME}...")
tokenized = dataset_dict.map(tokenize_fn, batched=True, batch_size=1000, remove_columns=['content'])
tokenized = tokenized.rename_column("label", "labels")

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# =====================================================
# 8. LOAD MODEL
# =====================================================
print(f"Đang tải model từ: {MODEL_NAME}...")
model = BertForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=2
)

# Cấu hình nhãn để dễ đọc khi inference sau này
model.config.id2label = {0: "Fake", 1: "Real"}
model.config.label2id = {"Fake": 0, "Real": 1}

# 9. TRAINING ARGUMENTS
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,
    warmup_ratio=0.1,
    weight_decay=0.01,
    learning_rate=3e-5,
    lr_scheduler_type="linear",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=100,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    fp16=torch.cuda.is_available(),
    dataloader_num_workers=2,
    report_to="none",
)

# 10. METRICS
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted", zero_division=0)
    acc = accuracy_score(labels, preds)
    try:
        probs = torch.softmax(torch.tensor(logits), dim=1).numpy()[:,1]
        auc = roc_auc_score(labels, probs)
    except:
        auc = None
    return {"accuracy": float(acc), "precision": float(precision), "recall": float(recall), "f1": float(f1), "auc": auc}

# 11. WEIGHTED TRAINER
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        weight_tensor = torch.tensor([class_weights[0], class_weights[1]], dtype=torch.float32, device=model.device)
        loss_fct = torch.nn.CrossEntropyLoss(weight=weight_tensor)
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

# 12. TRAINER
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
)

# 13. TRAIN
print("\n" + "="*60)
print("BẮT ĐẦU HUẤN LUYỆN – MINI-LM")
print("="*60)

trainer.train(resume_from_checkpoint=last_checkpoint)

manage_checkpoints(OUTPUT_DIR, keep_latest=2)

# 14. TEST
print("\nĐÁNH GIÁ TRÊN TẬP TEST")
results = trainer.evaluate(tokenized["test"])
for k, v in results.items():
    print(f"{k}: {v}")

# 15. SAVE MODEL
final_model_path = os.path.join(OUTPUT_DIR, "final_best_model")
trainer.save_model(final_model_path)
tokenizer.save_pretrained(final_model_path)

print(f"\nĐÃ LƯU MÔ HÌNH TỐT NHẤT TẠI:\n{final_model_path}")

# Cleanup
for f in os.listdir(OUTPUT_DIR):
    if f.startswith("checkpoint-") or f == "runs":
        shutil.rmtree(os.path.join(OUTPUT_DIR, f), ignore_errors=True)

print("Dọn dẹp hoàn tất. Chỉ giữ final model!")


Device: Tesla T4 | CUDA: True | VRAM: 15.8 GB
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive: 11.3GB / 16.1GB (70.4%)
Checkpoint gần nhất: Không có → Train từ đầu

Đang tải dataset LIAR (chengxuphd/liar2)...
Tổng số mẫu dữ liệu: 22962
Đang chuyển đổi nhãn sang Binary (0: Fake, 1: Real)...
Đang tạo nội dung đầu vào (Statement + Context)...
 → Sau xử lý: 22,962
Phân bố: Fake(0)=39.4%, Real(1)=60.6%
Class weights: {0: 1.2693200663349917, 1: 0.8249622763526622}
Train: 18,369 | Val: 2,296 | Test: 2,297


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

...


BẮT ĐẦU HUẤN LUYỆN – MINI-LM


model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Auc
1,0.662100,0.659372,0.617160,0.628077,0.617160,0.620641,0.652051
2,0.626800,0.650944,0.641115,0.640564,0.641115,0.640832,0.671740
3,0.602600,0.662813,0.620209,0.640322,0.620209,0.624521,0.671428



ĐÁNH GIÁ TRÊN TẬP TEST


eval_loss: 0.6610469222068787
eval_accuracy: 0.6351763169351328
eval_precision: 0.6313513185670545
eval_recall: 0.6351763169351328
eval_f1: 0.6328602594000667
eval_auc: 0.6586599828538768
eval_runtime: 2.6652
eval_samples_per_second: 861.859
eval_steps_per_second: 54.03
epoch: 3.0

ĐÃ LƯU MÔ HÌNH TỐT NHẤT TẠI:
/content/drive/MyDrive/WELFake_MiniLM_Pro/final_best_model
Dọn dẹp hoàn tất. Chỉ giữ final model!
